# 0. Env

In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm

import torch
from datasets import Dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline,
                          TrainingArguments)
from peft import (LoraConfig,
                  PeftModel)

# 1. Gemma-2b-it

In [ ]:
# 추론에 사용할 LLM
MODEL_ID = 'google/gemma-1.1-2b-it'
# hugging face access token을 복사하세요.
HF_TOKEN = ""

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)

In [ ]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          add_special_tokens=True,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

## 1.1. Tutorial

In [ ]:
# llm 추론 pipeline
# https://huggingface.co/docs/transformers/main_classes/pipelines
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

In [ ]:
doc = """엄청나게 즐거운 시간이었습니다. 강추!!!"""

In [ ]:
messages = [
    {
        "role": "user",
        "content": "다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)

In [ ]:
# gemma 기본 prompt 형식
print(prompt)

In [ ]:
# pipeline에 prompt를 입력해서 추론
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95
)
outputs

In [ ]:
# 전체 문장 출력
print(outputs[0]["generated_text"])

In [ ]:
# 프롬프트를 제외한 생성된 문장만 출력
print(outputs[0]["generated_text"][len(prompt):])

## 1.2. 연습문제
- 아래 gen_prompt를 수정해서 질문에 답하는 챗봇을 만들어 보세요.

In [ ]:
def gen_prompt(pipe, doc):
    messages = [
        {
            "role": "user",
            "content": "다음 문장은 영화리뷰입니다. 긍정 또는 부정으로 분류해주세요:\n\n{}".format(doc)
        }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)
    return prompt

In [ ]:
def gen_response(pipe, doc):
    prompt = gen_prompt(pipe, doc)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
while True:
    doc = input('문장 > ')
    doc = doc.strip()
    if len(doc) == 0:
        break
    result = gen_response(pipe, doc)
    print(f'감정 > {result}\n\n')